In [ ]:
from pynput import mouse, keyboard
import tkinter as tk
import pyautogui as pg
import time as t
import pyperclip
import random
import PIL, cv2
import keras
import tensorflow as tf
import numpy as np
import pandas as pd
from scipy import ndimage, misc
import os

pg.FAILSAFE = False
global graph, model

graph = tf.get_default_graph()

In [ ]:
"""ПРИМЕЧАНИЕ: Эта программа была создана для другого компьютера. Она не будет действовать для большинства мониторов. 
В принципе, это я баловался и не думал, что буду это кому-то показывать.

Программа использует нейросеть чтобы заходить на видео на Ютубе и определять, кликать ли на кнопку 'skip ads'.
Этим способом, можно оставлять компютер например на ночь, и программа будет давать просмотры для видео.

Также, можно эту программу использовать в качестве обычного авто-кликера. Это должно работать на любом компютере."""

"""This program is a small modification of the previous autoclicker app, which is specifically designed to give Youtube videos
views.

Sources on the internet suggest that a youtube video can be watched from the same IP/account 50 times for 50 separate views.
Each view can give the author of the video ad revenue provided that the user (1) watches the full advertisement,
(2) watches at least 30 seconds of a longer advertisement, or (3) interacts with the advertisement in some way.

Additionally, Youtube's algorithm may discard what it considers 'low-quality views'. This project is an attempt to write a script
that can get around these measures. Unfortunately, it must be noted that there is no easy way to learn if the program works or
is discarded by Youtube's algorithm, although such a thing could be accomplished by botting some anonymous channel with no views,
and hoping that no one else is watching it.

The autoclicker aspect of the program is used to navigate to a random video. Unlike many websites, Youtube's screen and 
'skip advertisement' buttons are not a part of simple HTML code. As a result, one can't click them using a simple webdriver such
as Selenium the same way one would normally navigate pages. Though there might be an easier way to accomplish the aforementioned
goal, this method will use an autoclicker to navigate to a certain website and wipe browser history each time.

Using a public VPN is a good idea to circumvent Youtube's count-freezing algorithm. Additionally, the browser history is wiped
each time to prevent Youtube from storing potential tracking cookies. 

The last part of this algorithm is a simple neural network. Because some advertisements can be very long or even require
a user to press 'skip ad' to play, a neural network will detect these elements on the screen and click accordingly. 
This will be done by simply screen-capping locations where the advertisement button could be (I don't know how to do a 
Region-of-Interest search) and running a small perceptron on each location. If an advertisement is found there, the program
will click it away after 30 seconds. 

Youtube tries to fool very basic bots by slightly shifting the location of its buttons and how they look, but a human cannot
detect these changes without paying close attention 
(ie. 'Skip Ads' could be re-written as 'Skip Ad' written in slightly darker text). This is why using a non-case sensitive
neural network makes sense. Additionally, the button can only appear within a small range of values, so we can run our algorithm
on images taken over that range.

The youtube video will be 'watched' by the bot for a random length of time, so as to not appear like a bot.

Unfortunately, this program is tailored to a certain screen size and locations of shortcuts and buttons on said screen. 
It will have to be modified to run on any other computer, by the user manually recording a clicking pattern and writing the
coordinates of where the ads might pop up.

The viewing program will run an infinite number of times (I usually use it when going to sleep). However, an autoclicker
program that has nothing to do with youtube or neural networks can also be used in this program.

"""

In [ ]:
#Contains a list to all relevant links, which we will visit for views.
videos = ['https://www.youtube.com/watch?v=XM2N-80wsOg', 'https://www.youtube.com/watch?v=PgllRJVwOvU', 'https://www.youtube.com/watch?v=Y0G7C985igQ',
          'https://www.youtube.com/watch?v=xTfTojfo3fI', 'https://www.youtube.com/watch?v=8XJCqYq6xXU', 'https://www.youtube.com/watch?v=F1WFcLUaOSE',
          'https://www.youtube.com/watch?v=BPTI8vhlvZ4', 'https://www.youtube.com/watch?v=hJPgDTVJzLU', 'https://www.youtube.com/watch?v=F_FnuAdnKCA',
          'https://www.youtube.com/watch?v=9PkNY-Zbmc8', 'https://www.youtube.com/watch?v=VxUVsfFh_hk', 'https://www.youtube.com/watch?v=LPyjtLdfNZk',
          'https://www.youtube.com/watch?v=pYN7nu4n07o', 'https://www.youtube.com/watch?v=ASh7KyCqUbo', 'https://www.youtube.com/watch?v=bgTKWIl_jyQ',
          'https://www.youtube.com/watch?v=_s-UxpCdCXw', 'https://www.youtube.com/watch?v=nyy9IelmtKU', 'https://www.youtube.com/watch?v=0-HKtpT0qWQ',
          'https://www.youtube.com/watch?v=XIoa4JfY9tw', 'https://www.youtube.com/watch?v=7Xqjk4GMthQ', 'https://www.youtube.com/watch?v=V50Y-aNe1yk',
          'https://www.youtube.com/watch?v=W5mhaQJw-qM', 'https://www.youtube.com/watch?v=kFmWaudUlOM', 'https://www.youtube.com/watch?v=Lsu1teoDmiU',
          'https://www.youtube.com/watch?v=qI16UU9wZgc', 'https://www.youtube.com/watch?v=HtjUsXexZlE', 'https://www.youtube.com/watch?v=KdtzzSpxjBA',
          'https://www.youtube.com/watch?v=Jae1n-BJSXk', 'https://www.youtube.com/watch?v=UncGwMF7qbU', 'https://www.youtube.com/watch?v=s2cNdyZ22Ec',
          'https://www.youtube.com/watch?v=_CEB6iZrSqQ', 'https://www.youtube.com/watch?v=8F6OWlcV9Kc', 'https://www.youtube.com/watch?v=XsY_GLzAT_A',
          'https://www.youtube.com/watch?v=JJ5aBH813cg', 'https://www.youtube.com/watch?v=aQZ6iCvtg3A', 'https://www.youtube.com/watch?v=aw0SfsIwLMo',
          'https://www.youtube.com/watch?v=NBjKNniIZ0s', 'https://www.youtube.com/watch?v=9yrFYq9EZrk', 'https://www.youtube.com/watch?v=jczNk2TRTdA',
          'https://www.youtube.com/watch?v=OvJowb0bI0c', 'https://www.youtube.com/watch?v=9b6iN1C5a3k', 'https://www.youtube.com/watch?v=QPTa4R1sFrk',
          'https://www.youtube.com/watch?v=ho9KEIyu-FM', 'https://www.youtube.com/watch?v=ryJ-oJRnCoY', 'https://www.youtube.com/watch?v=tUaWVaY7svM'
    
]

In [ ]:
#Helper function which is used to calculate the time before two actions.
def timetaken():
    global end; global start
    end = t.time()
    timetaken = max(end-start, start-end)
    start = t.time()
    return timetaken

#Helper function which is used to separate the main list of instructions and process into a set of sub-lists, each describing 
#an action and its details (type of action, time over which to complete action, position of the mouse, clicked or not clicked, etc.)
def listseparator(mainlist):
    mainlist = str(mainlist)
    separator = ','
    processedlist = []
    parts = ''
    mainlist += separator
    for i in mainlist:
        if (str(i) != separator):
            parts += str(i)
        else:
            processedlist.append(parts); parts = ''
    return processedlist
    

In [ ]:
#Initialize a bunch of necessary variables.
intro = "This is a simple autoclicker program, designed as a python mini-project. \n \nTo use this program, must first record a clicking pattern. To do this, press 'A' and map out your exact clicking pattern, and press 'A' again to stop recording. Then press 'B' to execute this clicking pattern as many times as you would like. \n\n Press 'C' to enter an optional waiting time before each clicking cycle is repeated. \n\nPress 'Q' or any other unrecognized hotkey to exit the program. \n\nClose this window to continue."  
instructions = []
time = 0
start = t.time()
Model = keras.models.load_model("C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/NNmodel")



In [ ]:
#Function to create the popup window, which contains the necessary instructions for the program.
def popup():
    root = tk.Tk()
    window = tk.Canvas(root, width = 1, height = 1)
    window.pack()
    button1 = tk.Button(root, text='Continue')
    window.create_window(250, 450, window=button1)
    Mytext = tk.Text(root, height=50, width=50)
    Mytext.pack()
    Mytext.insert(tk.END, intro)
  
    root.mainloop()

#Obtain a random link from the 'videos' variable and copy it to te clipboard
def Get_Link():
    random_link = random.choice(videos)
    pyperclip.copy(random_link)

#Read from the text file(s) to get the necessary directions for mouse movements
def Generate_Orders(*files):
    global processed_orders
    
    for afile in files:
        with open(afile) as file:
                orders = file.readlines()
                for i in orders:
                    i = str(i)
                    var = listseparator(i); processed_orders.append(var)
    
#Many variables are set to global so that they can be accessed by other functions.
#'file' is a variable loaded from a text document, to which the set of instructions are written to and loaded from.
#These instructions are loaded into a set of lists with the helper function 'listseparator()'
#'recording' is a variable which describes if the mouse movement is being recorded or not. Pressing 'a' toggles this.
def on_press(key):
    global recording; global file; global orders; global processed_orders; global start; global time
    
    timewait = 0
    if key.char == ('a'):
        if not recording:
            start = t.time()
            print("started recording."); recording = True
            file = open("instructions_list3.txt", mode = 'w')
        elif recording: 
            print("finished recording.")
            recording = False
            file.close()
            
    if key.char == ('c') and not recording:
        timewait = float(("Enter the amount of time you wish to wait after executing each click cycle: "))
        
    if key.char == ('b') and not recording:
        looptimes = input("Times to execute pattern: ")
        try: looptimes = int(looptimes)
        except: pass
        
        if (type(looptimes) is str):
            while True:
                t.sleep(5)
                Get_Link()
                act('instructions_list.txt')
                #We sleep for more than 30 seconds to provide time for the ad to load before playing
                t.sleep(40)
                #Since the 'skip ad' button tends to move, we cover all possible locations where it can exist.
                #20 arbitrary points are chosen, separated by 5 pixels along the vertical axis, and the network is run on each
                #image. This is a very ineffective way to do this and could be alleviated by training the neural network to
                #detect the button when it is displaced. Hypothetically, if the network was re-trained and some minor optimization
                #was done in terms of the regions covered by the scan, we could get away with running the network on 6 images or so
                #instead of 20. However, since this is a very simple and not resource-intensive algorithm, 
                #this workaround solution is deemed appropriate.
                for n in range(-10, 10):
                    if NeuralNetwork(area = [1795, 776+n*5, 1895, 820+n*5]):
                        pg.moveTo(1820, 796+n*5, 0.5); pg.click()
                Get_Link()
                act('instructions_list2.txt')
                t.sleep(30)
                for n in range(-10, 10):
                    if NeuralNetwork(area = [1795, 776+n*5, 1895, 820+n*5]):
                        pg.moveTo(1820, 796+n*5, 0.5); pg.click()
                time_watched = random.randint(30, 1000) + random.random()*100
                t.sleep(time_watched)
                
        else:
            for _ in range(looptimes):
                act('instructions_list3.txt')
                for i in processed_orders:
                    logic(i)
                    t.sleep(timewait)
    
    if key.char == ('q'): 
        global listener
        listener.stop()

#Run the neural network to determine if there is a clickable 'skip ad' button in the given area        
def NeuralNetwork(area = [1795, 866, 1895, 910]):
    global graph, Model
    try:
        img = Take_Picture(area)
        with graph.as_default():
            ans = np.argmax(Model.predict(img)[0])
        return ans
    except: 
        print('returning "False"')
        return False
#If the program is recording, every time a mouse is clicked or released the instruction is written to the text file.
#Commas are added to separate individual elements of an instruction as a list, 
#while new lines are used to separate individual instructions.
def on_click(x, y, button, pressed):
    global recording; global instructions; global file; global time
    if pressed is True and recording:
        print('Mouse clicked at ({0}, {1}) with {2}'.format(x, y, button)); time = timetaken()
        instructions = [x, y, button, time, 'pressed']
        for i in instructions:
            if i != instructions[-1]: file.write(str(i) + ',')
            else: file.write(str(i) + '\n')
    elif pressed is False and recording:
        print('Mouse released at ({0}, {1}) with {2}'.format(x, y, button)); time = timetaken()
        instructions = [x, y, button, time, 'released']
        for i in instructions:
            if i != instructions[-1]: file.write(str(i) + ',')
            else: file.write(str(i) + '\n')
#What happens each time the mouse wheel is scrolled.
def on_scroll(x, y, dx, dy):
    global recording; global file; global time
    if recording:
        print('Mouse scrolled at ({0}, {1})({2}, {3})'.format(x, y, dx, dy)); time = timetaken()
        instructions = [x, y, dx, dy, time, 'scrolling']
        for i in instructions:
            if i != instructions[-1]: file.write(str(i) + ',')
            else: file.write(str(i) + '\n')

#Only movement actions happening more than 0.1 seconds in between each other are recorded, so as to not lag the program
#when it executes the recorded steps.
def on_move(x, y):
    global time; time += timetaken()
    if recording and time > 0.1:
        instructions = [x, y, time, 'moving']
        for i in instructions:
            if i != instructions[-1]: file.write(str(i) + ',')
            else: file.write(str(i) + '\n')
        #Reset the time counter
        time = 0

#Function which executes instructions according to what is written in the aforementioned processed list of directions.
#By the chosen format, the type of action is listed as a string in the last variable of the list, so it is what 
#we are looking for in the [-1] element.
def logic(inputs):
    if 'pressed' in inputs[-1]:
        pg.moveTo(int(inputs[0]), int(inputs[1]), float(inputs[3]))
        if 'Button.left' in inputs[2]: pg.mouseDown(button = 'left')
        elif 'Button.right' in inputs[2]: pg.mouseDown(button = 'right')
    elif 'released' in inputs[-1]:
        pg.moveTo(int(inputs[0]), int(inputs[1]), float(inputs[3]))
        if 'Button.left' in inputs[2]: pg.mouseUp(button = 'left')
        elif 'Button.right' in inputs[2]: pg.mouseUp(button = 'right')
    elif 'scrolling' in inputs[-1]:
        pg.scroll(120*int(inputs[3]), int(inputs[1]), int(inputs[2]))
    elif 'moving' in inputs[-1]:
        pg.moveTo(int(inputs[0]), int(inputs[1]), float(inputs[2]))

#Act upon the processed_orders list
def act(instruction):
    global processed_orders
    processed_orders = []
    Generate_Orders(instruction)
    for i in processed_orders:
        logic(i)

In [ ]:
#Helper function to take pictures, and either save them to the harddrive for Neural Network training,
#or to help the network run when the program is looking for the 'skip ads' button.
def Take_Picture(corners, path = None):
    img = PIL.ImageGrab.grab(bbox=[corners])
    img = np.array(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.array(img)
    if path is not None:
        cv2.imwrite(path, img)

        img_orig = PIL.Image.fromarray(img)
        img_orig.show()
    if path is None:
        img = np.reshape(img, (1, corners[2] - corners[0], corners[3] - corners[1], 1))
        return img

In [ ]:
#Pop-up the instruction screen and set the recording to zero by default.
popup()
recording = False

#Join the mouse and keyboard from pynput into a single function.
def main():
    global listener
    with mouse.Listener(on_click=on_click, on_scroll=on_scroll, on_move = on_move) as listener:
        with keyboard.Listener(on_press=on_press) as listener:
            listener.join()
            
main()

In [ ]:
#Helper function which will create a dataframe and write the pixel-values of images to an excel file.
def create_dataframe(vertical_pixels, horizontal_pixels, writemode=True):
    label = np.matrix(1)
    images = []
    # Define the image size
    def getimages(directory):
        folder_length = 0
        for root, dirnames, filenames in os.walk(directory):
            # Search out individual files
            for filename in filenames:
                filepath = os.path.join(root, filename)
                # Resize the image, append to the images list
                image = ndimage.imread(filepath, mode="L")
                image_resized = misc.imresize(image, (horizontal_pixels, vertical_pixels))
                images.append(image_resized)
                folder_length += 1
        return folder_length
                
    length_ones = getimages("C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/images/1")
    length_twos = getimages("C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/images/0")

    # Define a matrix which will be mapped to.
    num_images = len(images)
    procimages = np.zeros(shape=(num_images, vertical_pixels * horizontal_pixels + 1))
    images = np.asarray(images)
    index = 0
    for i in images:
        if index == length_ones: label = np.matrix(0)
        # Reshape 3d pixel matrix into 2d matrix through flattening
        # concatenate the label with the reshaped matrix and map to procimages.
        j = np.reshape(i, (1, vertical_pixels * horizontal_pixels))
        k = np.concatenate([label, j], axis=1)
        procimages[index] = k
        index += 1

        # Convert to dataframe, and then write to excel file
    data = pd.DataFrame(procimages)
    if writemode is True:
        data.to_excel(excel_writer="C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/images.xlsx")
    else:
        with pd.ExcelWriter(excel_writer="C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/images.xlsx", engine="openpyxl",
                            mode="a") as writer:
            data.to_excel(writer, index=False, header=None)
        writer.save()
        
#A function which retrieves and filters information from an excel file for further processing.
def read_dataframe(path, horizontal_size, vertical_size):
    dataframe = pd.read_excel(path, header=None, index_col=None)
    dataframe = dataframe.sample(frac=1).reset_index(drop=True)
    dataframe = dataframe.as_matrix()
    
    dataframe = dataframe[:, 1:]
    dataframe = dataframe[1:, :]
    dataframe = np.nan_to_num(dataframe)
    
    X = dataframe[:, 0]
    Y = dataframe[:, 1:]
    X = [[1.0, 0.0] if i !=1 else [0.0, 1.0] for i in X]; X = np.array(X);
    
    ##Split data into test and train datasets.
    # Round the number so that the data can be split easily.
    tr_size = (round(np.size(X) / 4))
    
    X_test, X_train = X[0: tr_size - 1], X[tr_size:]
    Y_test, Y_train = Y[0:(tr_size - 1), :], Y[tr_size:, :]
    #Reshape the data
        
    Y_train, Y_test = np.reshape(Y_train, (-1, horizontal_size, vertical_size, 1)), np.reshape(Y_test, (-1, horizontal_size, vertical_size, 1))
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
t.sleep(5)
img = Take_Picture([1795, 866-55, 1895, 910-55])
#print(Model.predict(img))

In [ ]:
Breakb

In [ ]:
create_dataframe(100, 44, writemode = True)

In [ ]:
#Read the image pixel-values from the dataframe, labels and data, divided into train and test sets.
X_train, X_test, Y_train, Y_test = read_dataframe("C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/images.xlsx", 100, 44)

#Define the neural network
network = keras.Sequential()
network.add(keras.layers.Conv2D(50, kernel_size = (2, 2), strides = (1, 1), padding = 'same', input_shape = (100, 44, 1)))
network.add(keras.layers.Dropout(0.50))
network.add(keras.layers.Flatten())
network.add(keras.layers.core.Dense(50))
network.add(keras.layers.core.Dense(2, activation = 'softmax'))

In [ ]:
#Helper function to train and save our neural network
def train_network(model):
    optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, clipnorm=0.1)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    model.summary()
    history = model.fit(Y_train, X_train, batch_size=5, epochs=50)
    model.save("C:/Users/NS/Desktop/python stuff/projects/autoclicker/bot/NNmodel")

In [ ]:
train_network(network)

In [ ]:
#Testing some non-relevant stuff.
import time

def ntimes(n):
    def inner(f):
        def wrapper(*args, **kwargs):
            for _ in range(n):
                print(sum(args))
            rv = f(*args, **kwargs)
            return rv
        return wrapper
    return inner


@ntimes(6)
def add(x, y):
    return x + y

add(5, 6)

In [2]:
t.sleep(5)
print(pg.position())

Point(x=1919, y=818)


In [ ]:
bb